# 2.05: Blood Glucose Analysis
This notebook is used to analyse the blood glucose data from the patients. The intention is to understand blood glucose patterns and how they relate to changes in the insulin on board (IOB) and carbohydrates on board (COB). The analysis will focus on the following:
1. **Variability**: How much the blood glucose levels vary between nights, between individuals and for an individual.
2. **Rate of Change**: How quickly the blood glucose levels change over time.
3. **Peaks**: How often the blood glucose levels peak above a certain threshold for certain periods of the night.
4. **Correlation**: How the blood glucose levels correlate with the IOB and COB levels.
Lets start by loading the data and extracting aggregated data around variability for individuals.

In [10]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from src.configurations import Configuration
from src.data_processing.read_preprocessed_df import ReadPreprocessedDataFrame
from src.data_processing.read import read_profile_offsets_csv
from src.helper import separate_flat_file
from src.candidate_selection import remove_null_variable_individuals, \
    provide_data_statistics
from datetime import time
from src.data_processing.read_preprocessed_df import apply_and_filter_by_offsets
from src.time_series_analysis import return_count_intervals
from src.configurations import ThirtyMinute
from src.nights import Nights

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
config = Configuration()

df_new_30 = ReadPreprocessedDataFrame(ThirtyMinute(), file_type='parquet').df
df_offsets = read_profile_offsets_csv(config)
night_start = time(17, 0)
morning_end = time(11, 0)
sample_rate = 30
count_of_intervals = return_count_intervals(night_start, morning_end, minute_interval=sample_rate)
df_processed = apply_and_filter_by_offsets(offsets_df=df_offsets,
                                           interim_df=df_new_30)
df_processed = remove_null_variable_individuals(df_processed)
separated = separate_flat_file(df_processed)
df_overall_stats = provide_data_statistics(separated, sample_rate=sample_rate, night_start=night_start, morning_end=morning_end)


2025-06-23 22:05:17.202 | INFO     | src.data_processing.read_preprocessed_df:apply_and_filter_by_offsets:101 - IDs missing in offsets_df: {98974339, 35533061, 78420229, 66019205, 68026248, 40997757, 32997134, 1352464, 17161370, 49551394, 37998755, 40237051, 25692073, 74077367, 14092221, 18991425, 35719805, 43589707, 897741, 66937570, 65143138, 60844515, 27819368, 56958831, 80796147, 40634871, 16975609, 20396154, 81099003, 28176124, 86286077}
2025-06-23 22:05:17.751 | INFO     | src.candidate_selection:remove_null_variable_individuals:360 - Following individuals have one or more variables missing: set()
No stats per night have been calculated for 67167655. Returning no output.


The following analysis is based on the constraint (at the point of analysis) that we're putting on 'usable' nights which is that they have only a single 30min interval missing, or are complete, and a candidate must have 30 nights or greater. If these constraints change, then the analysis will only serve to show the process.

In [50]:
df_overall_stats['useful_nights'] = df_overall_stats['complete_nights'] + df_overall_stats['single_interval_nights']
df_stats = df_overall_stats[['count_of_nights', 'useful_nights', 'bg_sd_median', 'bg_range_median', 'bg_iqr_median']]
df_stats = df_stats[df_stats['useful_nights'] >= 30]
print(f'Total number of individuals with 30 or more useful nights: {len(df_stats)}')
print(f'Total number of nights across these individuals: {df_stats["useful_nights"].sum()}')

Total number of individuals with 30 or more useful nights: 17
Total number of nights across these individuals: 965
